<a href="https://colab.research.google.com/github/Maryamshahzadi-123/Quarter-2/blob/main/Project_01.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:

!pip install langchain google-cloud-aiplatform openai

from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from google.cloud import aiplatform

PROJECT_ID = "your-project-id"
LOCATION = "us-central1"
MODEL_ID = "projects/your-project-id/locations/us-central1/models/gemini-flash-2"

aiplatform.init(project=PROJECT_ID, location=LOCATION)

template = """You are a helpful assistant specializing in answering questions.
The user asks: "{question}"
Provide a concise and informative response."""

prompt = PromptTemplate(
    input_variables=["question"],
    template=template
)

class GeminiLLM(OpenAI):
    def __init__(self, project_id, model_id, temperature=0.7):
        self.project_id = project_id
        self.model_id = model_id
        self.temperature = temperature

    def call(self, prompt):
        client = aiplatform.gapic.PredictionServiceClient()
        endpoint = f"projects/{self.project_id}/locations/{LOCATION}/models/{self.model_id}"
        response = client.predict(
            endpoint=endpoint,
            instances=[{"content": prompt}],
            parameters={"temperature": self.temperature}
        )
        return response.predictions[0]['content']

gemini_llm = GeminiLLM(project_id=PROJECT_ID, model_id=MODEL_ID)

llm_chain = LLMChain(llm=gemini_llm, prompt=prompt)

questions = [
    "What are the benefits of AI in healthcare?",
    "How can businesses use social media for marketing?",
    "What is the importance of renewable energy?"
]

for question in questions:
    print(f"Question: {question}")
    print(f"Response: {llm_chain.run(question)}")
    print("-" * 50)

template_experiment = """You are an expert assistant. Provide detailed insights into the following query: "{question}"."""
prompt_experiment = PromptTemplate(
    input_variables=["question"],
    template=template_experiment
)

llm_chain_experiment = LLMChain(llm=gemini_llm, prompt=prompt_experiment)

print("\nExperimenting with a new prompt and parameters...")
for question in questions:
    print(f"Question: {question}")
    print(f"Response: {llm_chain_experiment.run(question)}")
    print("-" * 50)
